#### Model taken from:
- [1] Offshore Pipelaying Dynamics. Gullik Anthon Jensen 
- [2] A nonlinear PDE formulation for offshore vessel pipeline installation. Gullik A. Jensen et al 
- [3] Modeling and Control of Offshore Pipelay Operations Based on a Finite Strain Pipe Model. Gullik A. Jensen 

In [1]:
import numpy as np
import numdifftools as nd
import math

In [2]:
mp = 96 #  (submerged) [kg/m]

In [3]:
node_N=5

In [4]:
diag_Irho = 1e2*np.array([1, 1, 2]) # [m^4]  , p.99 in [1]
 
Irho=np.diag(diag_Irho)

In [5]:
Irho

array([[100.,   0.,   0.],
       [  0., 100.,   0.],
       [  0.,   0., 200.]])

In [6]:
qw = 1025 # Water density [kg/m3]
d0 = 0.761 # Outer diameter of pipe, [m]

In [7]:
diag_DT = 1.5*np.array([1, 1, 1]) # [N/m]  , p.99 in [1]

In [8]:
DT=np.diag(diag_DT) # (35) in [2]

In [9]:
DT

array([[1.5, 0. , 0. ],
       [0. , 1.5, 0. ],
       [0. , 0. , 1.5]])

In [10]:
diag_CT = 1e9*np.array([1, 1, 1]) # p. 4721 in [3]

In [11]:
CT=np.diag(diag_CT)

In [12]:
from sympy import *

In [13]:
# Dphi1, Dphi2, Dphi3 = symbols("Dphi1 Dphi2 Dphi3")
dx,dy,dz=np.array([.3 for i in range(node_N)]),np.array([.3 for i in range(node_N)]),np.array([.3 for i in range(node_N)]) 

In [14]:
def C1(dx,dy,dz):
    return np.vstack([dx**2,(dy**2+dz**2)**0.5*dy,(dy**2+dz**2)**0.5*dz])

In [15]:
C1(dx,dy,dz)

array([[0.09      , 0.09      , 0.09      , 0.09      , 0.09      ],
       [0.12727922, 0.12727922, 0.12727922, 0.12727922, 0.12727922],
       [0.12727922, 0.12727922, 0.12727922, 0.12727922, 0.12727922]])

In [16]:
1/2*d0*qw*np.dot(DT,np.array([0.09,0.12727922, 0.12727922])) 

array([52.6516875 , 74.46073019, 74.46073019])

In [17]:
def fD(dx,dy,dz,DT):
#     print(dx)
#     print(C1(dx,dy,dz).shape)
    return 1/2*d0*qw*np.dot(DT, C1(dx,dy,dz))

In [18]:
fD(dx,dy,dz,DT)

array([[52.6516875 , 52.6516875 , 52.6516875 , 52.6516875 , 52.6516875 ],
       [74.46073054, 74.46073054, 74.46073054, 74.46073054, 74.46073054],
       [74.46073054, 74.46073054, 74.46073054, 74.46073054, 74.46073054]])

In [19]:
# https://docs.scipy.org/doc/scipy-1.8.0/tutorial/integrate.html  
# 1-D Gray-Scott 

In [20]:
# Mat=np.matrix([[1.3,.4,6.7],[9,7,6],[3.3,5.6,7]])

In [21]:
I=np.identity(3)

In [22]:
np.dot(I,fD(dx,dy,dz,DT))

array([[52.6516875 , 52.6516875 , 52.6516875 , 52.6516875 , 52.6516875 ],
       [74.46073054, 74.46073054, 74.46073054, 74.46073054, 74.46073054],
       [74.46073054, 74.46073054, 74.46073054, 74.46073054, 74.46073054]])

In [23]:
Rb_t=I # https://www.researchgate.net/profile/Thor-Fossen/publication/224560837_Modeling_and_Control_of_Offshore_Pipelay_Operations_Based_on_a_Finite_Strain_Pipe_Model/links/00b7d520e175a3f918000000/Modeling-and-Control-of-Offshore-Pipelay-Operations-Based-on-a-Finite-Strain-Pipe-Model.pdf

In [24]:
Rb_t

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [25]:
diag_DR = 1.5*np.array([1, 1, 1]) #   p.4721 in [3]

In [26]:
DR=np.diag(diag_DR) 

In [27]:
dI= 0.69# Inner diameter of pipe, [m]

In [28]:
A=math.pi*((d0/2)**2-(dI/2)**2)

In [29]:
fg_e=np.array([[0,0,(mp-qw*A)*9.81] for x in range(node_N)]).reshape(3, node_N)

In [30]:
from numpy import linalg as LA

In [31]:
def k_sigma(hi,d0,fg_e):
    ans=[]
#     print('hi',hi)
    for it in hi:
        if it<0:
            ans.append(0)
        elif 0<=it<=d0/20:
#             print(LA.norm(fg_e,2)/(d0/8-d0/40)*10*it**2/d0)
            ans.append(LA.norm(fg_e,2)/(d0/8-d0/40)*10*it**2/d0)
        else:
#             print(LA.norm(fg_e,2)/(d0/8-d0/40)*10*hi**2/d0)
            ans.append(LA.norm(fg_e,2)/(d0/8-d0/40)*(it-d0/40))
#     print(ans)         
     
    return np.array(ans).reshape(node_N,1)   

In [32]:
x,y,z=np.array([2.3 for i in range(node_N)]),np.array([2.3 for i in range(node_N)]),np.array([2.3 for i in range(node_N)])

In [33]:
def hi(x,y,z,d0):
    return (np.dot(np.array([x,y,z]).T,[0,0,1])+d0/2).reshape(node_N,1) 

In [34]:
hi(x,y,z,d0).shape

(5, 1)

In [35]:
def sigma(x,y,z):
    return (k_sigma(hi(x,y,z,d0),d0,fg_e)*np.array([0,0,1])).reshape(3,node_N)

In [36]:
sigma(x,y,z).shape

(3, 5)

In [37]:
# angles should be in radians, otherwise np.radians them  
def Re_t_(arr):
    φ,θ,ψ=arr[0],arr[1],arr[2]
  
    Cφ=np.matrix([[1,0,0],
                      [0,np.cos(φ),-np.sin(φ)],
                      [0,np.sin(φ),np.cos(φ)]])

    Cθ=np.matrix([[np.cos(θ),0,np.sin(θ)],
                      [0,1,0],
                      [-np.sin(θ),0,np.cos(θ)]])

    Cψ=np.matrix([[np.cos(ψ),-np.sin(ψ),0],
                      [np.sin(ψ),np.cos(ψ),0],
                      [0,0,1]])
        

    return np.array(np.dot(np.dot(Cφ,Cθ),Cψ) )

In [38]:
Re_t_([1,2,3])

array([[ 0.41198225,  0.05872664,  0.90929743],
       [-0.68124272, -0.64287284,  0.35017549],
       [ 0.60512725, -0.76371834, -0.2248451 ]])

In [39]:
φ, θ, ψ = np.array([1 for i in range(node_N)]),np.array([2 for i in range(node_N)]),np.array([3 for i in range(node_N)])

In [40]:
def Re_t(a,b,c):
    xyz=np.stack((a,b,c), axis=1)
    return np.apply_along_axis(Re_t_, 1, xyz)

In [41]:
np.dot(Re_t(φ,θ,ψ)[0],Rb_t) 

array([[ 0.41198225,  0.05872664,  0.90929743],
       [-0.68124272, -0.64287284,  0.35017549],
       [ 0.60512725, -0.76371834, -0.2248451 ]])

In [42]:
# vessel motion
# Fossen book p.101, p.190, p.384, p.431
# Fossen paper, section 6

In [43]:
# Fossen paper, equation (12)

In [44]:
# φ,θ,ψ=1,2,3

In [45]:
def Re_b(φ,θ,ψ):
    return np.dot(Re_t(φ,θ,ψ), Rb_t ) # (5) in[3]

In [46]:
Re_b(φ,θ,ψ).shape

(5, 3, 3)

In [47]:
# p=np.dot(Re_b, phi) # (41) in[3]

In [48]:
def d_s(da,db,dc,dt,ds):
    return np.vstack([da*dt/ds,db*dt/ds,dc*dt/ds])

In [49]:
x,y,z=np.array([2.3 for i in range(node_N)]),np.array([.3 for i in range(node_N)]),np.array([8.3 for i in range(node_N)])
dt=0.1
ds=0.1

In [50]:
d_s(x,y,z,dt,ds).shape

(3, 5)

In [51]:
φ,θ,ψ=np.array([2.3 for i in range(node_N)]),np.array([1.3 for i in range(node_N)]),np.array([2.1 for i in range(node_N)])


In [52]:
CT

array([[1.e+09, 0.e+00, 0.e+00],
       [0.e+00, 1.e+09, 0.e+00],
       [0.e+00, 0.e+00, 1.e+09]])

In [53]:
def ne(x,y,z,φ,θ,ψ,dt,ds):
#     print(np.multiply(Re_t(φ,θ,ψ),CT).shape)
#     print(np.multiply(np.multiply(Re_t(φ,θ,ψ),CT),
#                          Re_t(φ,θ,ψ)).shape)
#     print(d_s(dx,dy,dz,dt,ds).shape)
    
    A=np.multiply(np.multiply(Re_t(φ,θ,ψ),CT),
                         Re_t(φ,θ,ψ))
    B=d_s(dx,dy,dz,dt,ds).T
#     print(A)
    
#     ans=[]
#     for i in range(len(A)):
#         ans.append(np.dot(A[i],B[i]))
    return np.einsum('ijk,ik->ij', A,B)
#     return np.array(ans)

In [54]:
def ne_(x,y,z,dx,dy,dz):
#     print(fg_e.shape)
#     print(fD(dx,dy,dz,DT).shape)
#     print(sigma(x,y,z).shape)
    return -fg_e-fD(dx,dy,dz,DT)-sigma(x,y,z)

In [55]:
# mp*ddp = d_s(ne,ds)+np.dot(Re_b,ne_)

In [56]:
dt=.01

In [57]:
ne(x,y,z,φ,θ,ψ,dt,ds)

array([[ 547120.56985966, 2417555.30585798,  952957.19016416],
       [ 547120.56985966, 2417555.30585798,  952957.19016416],
       [ 547120.56985966, 2417555.30585798,  952957.19016416],
       [ 547120.56985966, 2417555.30585798,  952957.19016416],
       [ 547120.56985966, 2417555.30585798,  952957.19016416]])

In [58]:
ne_(x,y,z,dx,dy,dz).shape

(3, 5)

In [59]:
Re_b(φ,θ,ψ).shape

(5, 3, 3)

In [60]:
def Irho_e(φ,θ,ψ):
    return np.multiply(np.multiply(Re_t(φ,θ,ψ),Irho),Re_t(φ,θ,ψ)) # (19) in [3] 

In [61]:
Irho_e(φ,θ,ψ).shape

(5, 3, 3)

In [62]:
def Π_(arr):
    φ,θ,ψ=arr[0],arr[1],arr[2]
    return np.array([[np.cos(θ)*np.cos(ψ),-np.sin(ψ),0],
                  [np.cos(θ)*np.sin(ψ),np.cos(ψ),0],
                  [-np.sin(θ),0,1]])

In [63]:
def Π(a,b,c):
    xyz=np.stack((a,b,c), axis=1)
  
    return np.apply_along_axis(Π_, 1, xyz)
    

In [64]:
Π(φ,θ,ψ).shape

(5, 3, 3)

In [65]:
def ωe(φ,θ,ψ):  # (100) in [2]
    A=Π(φ,θ,ψ)
#     print(A)
    B=np.array([φ,θ,ψ]).T
#     print(B)
#     ans=[]
#     for i in range(len(A)):
#         ans.append(np.dot(A[i],B[i]))
    return np.einsum('ijk,ik->ij', A,B)    
#     return np.array(ans) 
#     ans=[]
#     for i in range(len(φ)):
#         ans.append(np.dot(Π(φ[i],θ[i],ψ[i]),np.array([φ[i],θ[i],ψ[i]])))  # (100) in [2]
#     return np.vstack(ans)    

In [66]:
# np.dot(np.matrix([[-0.13504574, -0.86320937,  0.        ],
#   [ 0.23090749, -0.5048461 ,  0.        ],
#   [-0.96355819,  0.,          1.        ]]),np.array([2.3, 1.3, 2.1]))

In [67]:
ωe(φ,θ,ψ)

array([[-1.43277738, -0.1252127 , -0.11618383],
       [-1.43277738, -0.1252127 , -0.11618383],
       [-1.43277738, -0.1252127 , -0.11618383],
       [-1.43277738, -0.1252127 , -0.11618383],
       [-1.43277738, -0.1252127 , -0.11618383]])

In [68]:
diag_CR = 1e11*np.array([1, 1, 1]) 
 
CR=np.diag(diag_CR)

In [69]:
def me(φ,θ,ψ):
    A=np.multiply(np.multiply(Re_t(φ,θ,ψ),CR),Re_t(φ,θ,ψ))
    B=ωe(φ,θ,ψ)
    ans=[]
    return np.einsum('ijk,ik->ij', A,B) 
#     for i in range(len(A)):
#         ans.append(np.dot(A[i],B[i]))
#     return np.array(ans)
#     tmp=[]
#     for i in range(len(φ)):
#         tmp.append(np.dot(np.dot(Re_t(φ[i],θ[i],ψ[i]),CR),Re_t(φ[i],θ[i],ψ[i]).T))
        
#     tmp1= ωe(φ,θ,ψ)
    
#     ans=[]
#     for i in range(len(tmp1)):
#         ans.append(np.dot(np.array(tmp[i]),tmp1[i] ))
#     return np.array(ans).reshape(3,-1)

In [70]:
me_=me(φ,θ,ψ)

In [71]:
me_

array([[-2.61300659e+09, -1.00902875e+09, -3.69060709e+08],
       [-2.61300659e+09, -1.00902875e+09, -3.69060709e+08],
       [-2.61300659e+09, -1.00902875e+09, -3.69060709e+08],
       [-2.61300659e+09, -1.00902875e+09, -3.69060709e+08],
       [-2.61300659e+09, -1.00902875e+09, -3.69060709e+08]])

In [72]:
diag_DR = 1.5*np.array([1, 1, 1]) 
 
DR=np.diag(diag_DR)

In [73]:
# np.dot(Re_b, np.dot( Irho_e, (np.dot(Π(φ,θ,ψ), np.array([ddφ,ddθ,ddψ])))))=
# -np.dot(Re_b,np.dot(Irho_e,np.dot(Π(dφ,dθ,dψ),np.array([dφ,dθ,dψ])))-
# np.cross(np.dot(Π(φ,θ,ψ),np.array([dφ,dθ,dψ])),np.dot(Irho_e, np.dot(Π(φ,θ,ψ),np.array([dφ,dθ,dψ]))))+
# np.dot(Re_b,d_s(me_[0],me_[1],me_[2],dt,ds)+d_s(me_[0],me_[1],me_[2],dt,ds))+
# np.cross(d_s(x,y,z,dt,ds),ne(x,y,z,φ,θ,ψ,dt,ds))+
# np.dot(Re_b,-np.dot(DR,np.dot(Π(φ,θ,ψ),np.array([dφ,dθ,dψ]))))

In [74]:
ne_(x,y,z,dx,dy,dz).shape

(3, 5)

In [75]:
C=ne(x,y,z,φ,θ,ψ,dt,ds)
  
a,b,c=C[:,0],C[:,1],C[:,2]

In [76]:
d_s(a,b,c,dt,ds).shape

(3, 5)

In [77]:
Re_b(φ,θ,ψ).shape

(5, 3, 3)

In [78]:
ne_(x,y,z,dx,dy,dz).T

array([[   -52.6516875 , -20832.19110425,    -74.46073054],
       [   -52.6516875 ,    -74.46073054, -20832.19110425],
       [-20810.3820612 ,    -74.46073054,    -74.46073054],
       [   -52.6516875 , -20832.19110425,    -74.46073054],
       [   -52.6516875 ,    -74.46073054, -20832.19110425]])

In [79]:
type(x),y,z,dx,type(dy),dz

(numpy.ndarray,
 array([0.3, 0.3, 0.3, 0.3, 0.3]),
 array([8.3, 8.3, 8.3, 8.3, 8.3]),
 array([0.3, 0.3, 0.3, 0.3, 0.3]),
 numpy.ndarray,
 array([0.3, 0.3, 0.3, 0.3, 0.3]))

In [80]:
ne_(x,y,z,dx,dy,dz).T+np.einsum('ijk,ik->ij', Re_b(φ,θ,ψ),ne_(x,y,z,dx,dy,dz).T)

array([[  4693.02050378, -14854.21274363,  19309.31787368],
       [-20101.37601926,   4151.56491178, -17066.84082255],
       [-18054.58228812,  19479.23255169,  -6642.69770626],
       [  4693.02050378, -14854.21274363,  19309.31787368],
       [-20101.37601926,   4151.56491178, -17066.84082255]])

In [81]:
def grayscott1d(Q, t, ds):
#     print(type(Q))
    x,y,z=Q[0::12],Q[2::12],Q[4::12]
    dx,dy,dz=Q[1::12],Q[3::12],Q[5::12]
#     print(dx,dy.shape, dx.shape)
    φ,θ,ψ=Q[6::12],Q[8::12],Q[10::12]
    dφ,dθ,dψ=Q[7::12],Q[9::12],Q[11::12]
    
    
    ddx = np.empty_like(y)
    ddy = np.empty_like(y)
    ddz = np.empty_like(y)
    ddφ = np.empty_like(y)
    ddθ = np.empty_like(y)
    ddψ= np.empty_like(y)
    C=ne(x,y,z,φ,θ,ψ,dt,ds)
  
    a,b,c=C[:,0],C[:,1],C[:,2]
#     print(a)
    
#     for it in [φ,θ,ψ,x,y,z,dx,dy,dz]:
#         print(it)
    # here single Re_b(φ,θ,ψ) etc.
#     print(np.einsum('ijk,ik->ij', Re_b(φ,θ,ψ).astype(float), ne_(x,y,z,dx,dy,dz).astype(float).T))
    ddx,ddy, ddz = (1/mp*(d_s(a,b,c,dt,ds).T + 
                          np.einsum('ijk,ik->ij', Re_b(φ,θ,ψ).astype(float), ne_(x,y,z,dx,dy,dz).astype(float).T)
                         )).T
#     print('ddx',ddx)                
    
    me_=me(φ,θ,ψ)
#     print(Re_b(φ,θ,ψ).shape)
#     print(Irho_e(φ,θ,ψ).shape)
#     print(Irho_e(φ,θ,ψ).shape)

    
    C1=np.einsum('ijk,ik->ij', Re_b(φ,θ,ψ).astype(float), np.einsum('ijk,ik->ij', Irho_e(φ,θ,ψ).astype(float),
                                                        np.einsum('ijk,ik->ij', Π(dφ,dθ,dψ).astype(float), np.array([dφ,dθ,dψ]).astype(float).T)))
        
    C2= np.cross(np.einsum('ijk,ik->ij',Π(φ,θ,ψ).astype(float),np.array([dφ,dθ,dψ]).astype(float).T),
                 np.einsum('ijk,ik->ij',Irho_e(φ,θ,ψ).astype(float),
                           np.einsum('ijk,ik->ij',Π(φ,θ,ψ).astype(float),np.array([dφ,dθ,dψ]).astype(float).T)))
    
#     print(Re_b(φ,θ,ψ).shape)
# #     print(me_)
# #     print(me_[:,0])
#     print(d_s(me_[:,0],me_[:,1],me_[:,2],dt,ds).shape)
# #     print(Irho_e(φ,θ,ψ).shape)
    C3= np.einsum('ijk,ik->ij',Re_b(φ,θ,ψ).astype(float),d_s(me_[:,0],me_[:,1],me_[:, 2],dt,ds).astype(float).T)
    
#     print(d_s(x,y,z,dt,ds).shape)
#     print(ne(x,y,z,φ,θ,ψ,dt,ds).T.shape)
    C4= np.cross(d_s(x,y,z,dt,ds).T,ne(x,y,z,φ,θ,ψ,dt,ds))
#     print(np.dot(DR,K).shape)
#     print(Re_b(φ,θ,ψ).shape)
#     print(np.dot(DR,K[:,0]))
    K1=np.einsum('ijk,ik->ij',Π(φ,θ,ψ).astype(float),np.array([dφ,dθ,dψ]).astype(float).T).T
    C5= np.einsum('ijk,ik->ij',Re_b(φ,θ,ψ).astype(float),-np.dot(DR,K1).astype(float).T)
#     print(C1.shape)
#     print(C2.shape)
#     print(C3.shape)
#     print(C4.shape)
#     print(C5.shape)
    B =-C1-C2+C3+C4+C5
#     print(Re_b(φ,θ,ψ).shape)
#     print(Irho_e(φ,θ,ψ).shape)
#     print(Π(φ,θ,ψ).shape)
    A1 =Re_b(φ,θ,ψ).astype(float)
    A2 = Irho_e(φ,θ,ψ).astype(float)
    A3=Π(φ,θ,ψ).astype(float)
#     print(np.einsum('ijk,ijk->ijk',A1,A2))
#     print('A1',A1)
#     print(A1)
#     print('bu',A1[0])
#     print('hey', np.dot(A1[0],A2[0]))
#     print('op', np.einsum('ijk,ikr->ijr',A1,A2))
#     print(Π(φ,θ,ψ).shape)
    A=np.einsum('ijk,ikr->ijr',np.einsum('ijk,ikr->ijr',A1,A2),A3)
#     print(A.shape)
#     print(B.shape)
#     print(np.linalg.solve(A,B) ) 
    ddφ,ddθ,ddψ = np.linalg.solve(A,B).T 
#     print(ddφ)
#     print(np.concatenate([dx, ddx, dy, ddy, dz, ddz, dφ, ddφ, dθ, ddθ, dψ, ddψ]))
    
    return np.concatenate([dx, ddx, dy, ddy, dz, ddz, dφ, ddφ, dθ, ddθ, dψ, ddψ])
    

In [82]:
rng = np.random.default_rng()
q0=rng.standard_normal(12*node_N)

In [83]:
# q0=[i for i in range(12)]
max_t = 10  
dt = 0.5
t=np.arange(0,max_t,dt)
ds = 0.025
print(len(t))

20


In [84]:
from datetime import datetime
from scipy.integrate import odeint

In [85]:
startTime1 = datetime.now()
us=odeint(grayscott1d, q0, t, args=(ds,))
print(datetime.now() - startTime1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  from ipykernel import kernelapp as app


0:00:31.452279


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [86]:
us.shape

(20, 60)

In [87]:
us

array([[ 1.98392767e+00,  1.79674291e+00,  4.83324650e-02, ...,
        -1.04440054e+00,  1.00531324e+00, -2.66617204e+00],
       [ 1.98392767e+00,  1.79674291e+00,  4.83324650e-02, ...,
        -1.15079180e+00,  1.26673244e+00,  6.97538239e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [ ]:
def psi(gamma,omega,Ct,Cr):
    coord=np.concatenate((gamma, omega), axis=0)
    zero=np.zeros((3,3))
    M1=np.concatenate((Ct, zero), axis=0)
    M2=np.concatenate((zero, Cr), axis=0)
    M=np.concatenate((M1, M2), axis=1)
    return 1/2*np.dot(np.dot(coord.T,M),coord)

In [ ]:
from sympy import *

In [ ]:
gamma1, gamma2,gamma3 = symbols("gamma1 gamma2 gamma3")
omega1, omega2,omega3 = symbols("omega1 omega2 omega3")

In [ ]:
gamma=np.array([gamma1, gamma2,gamma3])
omega=np.array([omega1, omega2,omega3])

In [ ]:
Ct=np.matrix([[1,0,0],[0,2,0],[0,0,3]])
Cr=np.matrix([[4,0,0],[0,5,0],[0,0,6]])

In [ ]:
psi(gamma,omega,Ct,Cr)

In [ ]:
fun = lambda x, y: psi(x,y,Ct,Cr)

In [ ]:
Dfun = nd.Gradient(fun)

In [ ]:
Dfun()

In [ ]:
# coord=np.concatenate((gamma, omega), axis=0)

In [ ]:
gamma

In [ ]:
omega

In [ ]:
Dfun(gamma,omega)

In [ ]:
Dfun